In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [3]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex2"
today

'day17'

In [4]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [5]:
with open(f"./data/{today}.txt") as f:
    lines = f.read()

A_init = parse_nums(lines.split("\n")[0].rstrip())[0]
B_init = parse_nums(lines.split("\n")[1].rstrip())[0]
C_init = parse_nums(lines.split("\n")[2].rstrip())[0]
program_init = parse_nums(lines.split("\n")[4].rstrip())

len(program_init)

16

In [6]:
def combo(operand: int, A: int, B: int, C: int) -> int:
    match operand:
        case 0 | 1 | 2 | 3:
            return operand
        case 4:
            return A
        case 5:
            return B
        case 6:
            return C
        case _:
            raise Exception(f"Invalid combo operand: {operand}")

In [7]:
def run_program(A: int, B: int, C: int, program: list[int], find_orig: bool = False) -> list[int]:
    output = []
    i = 0
    while i < len(program):
        opcode = program[i]
        operand = program[i+1]

        match opcode:
            case 0:
                val = combo(operand, A, B, C)
                A = A // 2**val
            case 1:
                B = B ^ operand
            case 2:
                val = combo(operand, A, B, C)
                B = val % 8
            case 3:
                if A != 0:
                    i = operand - 2
            case 4:
                B = B ^ C
            case 5:
                val = combo(operand, A, B, C)
                output.append(val % 8)
                if find_orig and (len(output) > len(program) or output[-1] != program[len(output)-1]):
                    return []
            case 6:
                val = combo(operand, A, B, C)
                B = A // 2**val
            case 7:
                val = combo(operand, A, B, C)
                C = A // 2**val
            case _:
                raise Exception(f"boom: {i} {opcode} {operand} {A} {B} {C}")
        i += 2
    return output

In [8]:
o = run_program(A_init, B_init, C_init, program_init.copy())
print(",".join([str(x) for x in o]))

6,5,4,7,1,6,0,3,1


In [9]:
def run_program_2(A: int, find_orig: bool = False) -> list[int]:
    output = []
    while A > 0:
        output.append(((A % 8) ^ 3) ^ (A // (2**((A % 8) ^ 5))) % 8)
        A //= 8
        if find_orig and (len(output) > len(program_init) or output[-1] != program_init[len(output)-1]):
            return []
    return output

In [10]:
run_program_2(44348299)

[6, 5, 4, 7, 1, 6, 0, 3, 1]